In [13]:
from collections import Counter
from django.core.exceptions import ObjectDoesNotExist, FieldError
import pandas as pd
import re
from import_utils import utils

In [14]:
col, _ = Collection.objects.get_or_create(name="default")
file = "entities/data/vb_st_michaelsburg.xlsx"

In [25]:
dfs = utils.get_dfs(file, filterstring='V')
len(dfs)

51

In [26]:
# import easy values
# create basic vocabs and relations
ww_kind, _ = WorkWorkRelation.objects.get_or_create(
    name='ist Teil von'
)
ww_kind.name_reverse = 'beinhaltet'
ww_kind.save()
sla, _ = Institution.objects.get_or_create(
    name='Südtiroler Landesarchiv'
)
sla_vb_kind, _ = InstitutionWorkRelation.objects.get_or_create(
    name = 'archiviert'
)
sla_vb_kind.name_reverse = 'ist archiviert in'
sla_vb_kind.save()
for df in dfs:
    vb_name = df.loc[0][0].split(',')[0]
    vb_date = "{}".format([int(s) for s in vb_name.split() if s.isdigit()][0])
    vb, _ = Work.objects.get_or_create(
        name=vb_name
    )
    vb_kind, _ = WorkType.objects.get_or_create(
            name='Verfachbuch'
        )
    vb.kind = vb_kind
    vb.start_date_written = vb_date
    vb.end_date_written = "{}-12-31".format(vb_date)
    vb.save()
    sla_vb, _ = InstitutionWork.objects.get_or_create(
        relation_type=sla_vb_kind,
        related_institution=sla,
        related_work=vb
    )
    for i, row in df.iterrows():
        if row['Signatur'] == 'FALSE':
            temp_work = None
        else:
            temp_work, _ = Work.objects.get_or_create(
                written_name=row['Signatur']
            )
            temp_work.name = row['Signatur']
        
        if temp_work:
        
    # add start and end date
            if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
                pass
            else:
                date_string_list = row['Datum der Amtshandlung'].split('\n')
                dates = utils.get_start_end(date_string_list)
                if dates:
                    if len(dates) > 1:
                        temp_work.start_date_written, temp_work.end_date_written = dates
                        temp_work.save()
                    else:
                        temp_work.start_date.written = dates[0]
                        temp_work.save()

            if row['Buch/Bücher'] == 'FALSE' or row['Buch/Bücher'] == 'n. a.':
                temp_work.buecher = ""
            else:
                temp_work.buecher = row['Buch/Bücher']
            if row['Summe Inventar'] == 'FALSE' or row['Summe Inventar'] == 'n. a.':
                temp_work.summe_inventar = ""
            else:
                temp_work.summe_inventar = row['Summe Inventar']
            if row['Barschaft (teilweise eigene Berechnung)'] == 'FALSE' or row['Barschaft (teilweise eigene Berechnung)'] == 'n.a.':
                temp_work.barschaft= ""
            else:
                temp_work.barschaft = row['Barschaft (teilweise eigene Berechnung)']
            temp_work.save()
        # kind
            raw_kind = row['Dokumententypus'].split('\n')[0].split(';')[0]
            work_kind, _ = WorkType.objects.get_or_create(
                name=raw_kind
            )
            temp_work.kind = work_kind
            temp_work.save()
        # link to vb
            temp_work_ww, _ = WorkWork.objects.get_or_create(
                relation_type=ww_kind,
                related_workA=temp_work,
                related_workB=vb
            )

        # add start and end date
            if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
                pass
            else:
                date_string_list = row['Datum der Amtshandlung'].split('\n')
                dates = utils.get_start_end(date_string_list)
                if dates:
                    if len(dates) > 1:
                        temp_work_ww.start_date_written, temp_work_ww.end_date_written = dates
                        temp_work_ww.save()
                    else:
                        temp_work_ww.start_date.written = dates[0]
                        temp_work_ww.save()
                else:
                    pass

In [31]:
# add comments
temp_work = None
for df in dfs:
    for i, row in df.iterrows():
        try:
            temp_work = Work.objects.get(written_name=row['Signatur'])
        except ObjectDoesNotExist:
            temp_work = None
        
        if temp_work:
            if row['comments b'] != 'FALSE':
                temp_work.comment_b = row['comments b']
            if row['comments a'] != 'FALSE':
                temp_work.comment_a = row['comments a']
            if row['comments k'] != 'FALSE':
                temp_work.comment_a = row['comments k']
            try:
                temp_work.save()
            except:
                print(temp_work)
            

In [32]:
# add info about money and main persons
temp_work = None
for df in dfs:
    for i, row in df.iterrows():
        temp_work = None
        try:
            temp_work = Work.objects.get(written_name=row['Signatur'])
        except ObjectDoesNotExist:
            temp_work = None
        if temp_work:
            
            if row['gender'] == -4142:
                temp_work.only_one_person = 'mehrere Personen'
                temp_work.save()
            if row['gender'] == 33:
                temp_work.only_one_person = 'nur eine Hauptperson'
                temp_work.save()
            if row['gender'] == 3:
                temp_work.only_one_person = 'nur eine Hauptperson'
                temp_work.save()
            try:
                temp_work.save()
            except:
                print(temp_work)
        
            if isinstance(row['Gulden Inventarsumme'], int) or isinstance(row['Gulden Inventarsumme'], float):
                temp_work.invenatar_summe_norm_fl = int(row['Gulden Inventarsumme'])
            if isinstance(row['Kreuzer Inventarsumme'], float) or isinstance(row['Kreuzer Inventarsumme'], int):
                temp_work.invenatar_summe_norm_kr = row['Kreuzer Inventarsumme']

            if isinstance(row['Gulden vor Abzug Passiva'], int) or isinstance(row['Gulden vor Abzug Passiva'], float):
                temp_work.vor_passiva_fl = int(row['Gulden vor Abzug Passiva'])            
            if isinstance(row['Kreuzer vor Abzug Passiva'], float) or isinstance(row['Kreuzer vor Abzug Passiva'], int):
                temp_work.vor_passiva_kr = row['Kreuzer vor Abzug Passiva']

            if isinstance(row['Gulden nach Abzug Passiva'], int) or isinstance(row['Gulden nach Abzug Passiva'], float):
                temp_work.nach_passiva_fl = int(row['Gulden nach Abzug Passiva'])            
            if isinstance(row['Kreuzer nach Abzug Passiva'], float) or isinstance(row['Kreuzer nach Abzug Passiva'], int):
                temp_work.nach_passiva_kr = row['Kreuzer vor Abzug Passiva']

            try:
                temp_work.save()
            except:
                print(temp_work)

Verfachbuch St. Michaelsburg 1755, Bl. 85 r-91 r
Verfachbuch St. Michaelsburg 1781, Bl. 456 r-461 v
Verfachbuch St. Michaelsburg 1788, Bl. 113 r-118 v
Verfachbuch St. Michaelsburg 1791, Bl. 302 r-351 v


In [33]:
# Hauptpersonen, deren Wohnorte, jobs, geschlecht
temp_work = None
place = None
parent_pers_work = None
parent_pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="wird erwähnt", name_reverse='erwähnt'
)
pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name='geführt als Hauptperson', name_reverse='führt als Hauptperson'
)
pers_work.parent_class = parent_pers_work
pers_work.save()
pers_place, _ = PersonPlaceRelation.objects.get_or_create(
    name='lebte in', name_reverse='Wohnort von'
)
for df in dfs:
    for i, row in df.iterrows():
        try:
            temp_work = Work.objects.get(written_name=row['Signatur'])
        except ObjectDoesNotExist:
            temp_work = None
            
        if temp_work:
            place = utils.create_places_from_place_string(
                row['Wohnort; Ort des betreffenden Gutes/der betreffenden Behausung'].split('\n')[0]
            )
            all_names = row['Name\n(Erklärung aus Verfachbuch)'].split('\n')[0]
            job = row['Beruf'].strip().split('\n')[0]
            if job.startswith('(') or job.startswith('n.') or ":" in job or ";" in job:
                job_obj = None
            else:
                job = job.replace('"', "")
                job_obj, _ = ProfessionType.objects.get_or_create(
                    name=job
                )
            # check for color and only if no color then split by ; 
                for written_name in all_names.split(';'):
                    written_name = written_name.strip()
                    pers = None
                    pers, _ = Person.objects.get_or_create(
                        written_name=written_name
                    )
                    pers.name=written_name
                    pers.first_name = written_name.split(' ')[0]

                    pers_work_rel, _ = PersonWork.objects.get_or_create(
                        relation_type=pers_work,
                        related_person=pers,
                        related_work=temp_work
                    )

                # add start and end date
                    if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
                        pass
                    else:
                        date_string_list = row['Datum der Amtshandlung'].split('\n')
                        dates = utils.get_start_end(date_string_list)
                        if dates:
                            if len(dates) > 1:
                                pers_work_rel.start_date_written, pers_work_rel.end_date_written = dates
                                pers_work_rel.save()
                            else:
                                pers_work_rel.start_date.written = dates[0]
                                pers_work_rel.save()

                # person-place relation
                    pers_pl_rel, _ = PersonPlace.objects.get_or_create(
                        relation_type=pers_place,
                        related_person=pers,
                        related_place=place[0]
                    )
                    if job_obj:
                        pers.profession.add(job_obj)

                # gender
                    if row['Signatur'] == 'FALSE':
    #                     print('FALSE')
                        pass
                    else:
    #                     print(type(row['gender']))
                        if row['gender'] == -4142:
                            pass
                        elif row['gender'] == 3:
                            pers.gender = 'female'
                            pers.save()
                        elif row['gender'] == 33:
                            pers.gender = 'male'
                            pers.save()
                    pers.save()

In [ ]:
# add passive beteiligte Personen
temp_work = None
place = None
parent_pers_work = None
pers_work = None
parent_pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="wird erwähnt", name_reverse='erwähnt'
)
pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="als passiv beteilligte Personen erwähnt", name_reverse='erwähnt als passiv beteilligte Person'
)
pers_work.parent_class = parent_pers_work
pers_work.save()
all_names = None
for df in dfs:
    for i, row in df.iterrows():
        all_names = row['Beteiligte Personen (Erbsinteressenten, Gerhaben, Anweiser, Verkäufer, Verpächter, Käufer, Pächter, ...)'].split('\n')
        temp_work = Work.objects.get(written_name=row['Signatur'])
        for written_name in all_names:
            pers = None
            written_name = written_name.strip()
            pers, _ = Person.objects.get_or_create(
                written_name=written_name
            )
            pers.name=written_name
            pers.first_name = written_name.split(' ')[0]
            pers_work_rel, _ = PersonWork.objects.get_or_create(
                relation_type=pers_work,
                related_person=pers,
                related_work=temp_work
            )

        # add start and end date
            if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
                pass
            else:
                date_string_list = row['Datum der Amtshandlung'].split('\n')
                dates = utils.get_start_end(date_string_list)
                if dates:
                    if len(dates) > 1:
                        pers_work_rel.start_date_written, pers_work_rel.end_date_written = dates
                        pers_work_rel.save()
                    else:
                        pers_work_rel.start_date.written = dates[0]
                        pers_work_rel.save()
            pers.save()

In [ ]:
# add aktiv beteiligte Personen
temp_work = None
place = None
parent_pers_work = None
pers_work = None
parent_pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="wird erwähnt", name_reverse='erwähnt'
)
pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="als aktiv beteilligte Personen erwähnt", name_reverse='erwähnt als aktiv beteilligte Person'
)
pers_work.parent_class = parent_pers_work
pers_work.save()
all_names = None
for df in dfs:
    for i, row in df.iterrows():
        all_names = row['Beteiligte Personen (Beamte, Gerichtsverpflichtete, Zeugen, ...)'].split('\n')
        temp_work = Work.objects.get(written_name=row['Signatur'])
        for written_name in all_names:
            pers = None
            written_name = written_name.strip()
            if written_name.endswith(':'):
                pass
            else:
                pers, _ = Person.objects.get_or_create(
                written_name=written_name
                )
                pers.name=written_name
                pers.first_name = written_name.split(' ')[0]
                pers.save()
                pers_work_rel, _ = PersonWork.objects.get_or_create(
                        relation_type=pers_work,
                        related_person=pers,
                        related_work=temp_work
                )
            
            # add start and end date
                if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
                    pass
                else:
                    date_string_list = row['Datum der Amtshandlung'].split('\n')
                    dates = utils.get_start_end(date_string_list)
                    if dates:
                        if len(dates) > 1:
                            pers_work_rel.start_date_written, pers_work_rel.end_date_written = dates
                            pers_work_rel.save()
                        else:
                            pers_work_rel.start_date.written = dates[0]
                            pers_work_rel.save()

In [ ]:
for x in Work.objects.filter(buecher__startswith='j'):
    x.buecher_sys = "Bücher"
    x.save()

In [ ]:
for x in Work.objects.filter(buecher__contains='KEIN'):
    x.vollstaendig = "unvollständig"
    x.save()

In [ ]:
for x in Work.objects.filter(buecher__icontains='allerdings'):
    x.schreib_sachen = "Lese- und Schreibsachen"
    x.save()

In [11]:
Work.objects.all().delete()

(903,
 {'labels.Label': 0,
  'metainfo.Uri': 121,
  'metainfo.UriCandidate': 0,
  'entities.Work': 121,
  'relations.PersonWork': 149,
  'relations.InstitutionWork': 6,
  'relations.WorkWork': 115,
  'metainfo.TempEntityClass': 391})

In [12]:
Person.objects.all().delete()

(786,
 {'labels.Label': 0,
  'metainfo.Uri': 144,
  'metainfo.UriCandidate': 0,
  'entities.Person_profession': 62,
  'relations.PersonPlace': 146,
  'entities.Person': 144,
  'metainfo.TempEntityClass': 290})